# LOCALIZACIONES DEFINIDAS
posibles lugares bien situados

In [1]:
london_loc = "51.520624794291884, -0.0870022262169723"

In [2]:
singapore_loc = "1.2862863534171092, 103.84868192263733"

In [3]:
san_francisco_loc = "37.79787988216447, -122.41049370727525"

# API
importar librerias

In [4]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import pyjsonviewer
import geopandas 
import shapely
import pymongo as pym
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry as shg

# FUNCIONES DE LLAMADA A LA API

In [5]:
load_dotenv()
#Tenemos los tokens guardados en un .env en el visual studio code

True

In [6]:
tk_id = os.getenv("token_client_id")

In [7]:
tk_secret = os.getenv("token_client_secret")

In [8]:
git_tk = os.getenv("github_tk")

In [9]:
url = 'https://api.foursquare.com/v2/venues/explore'

In [10]:
#pyjsonviewer.view_data(json_data = LaVariableQueQuierasSinComillas)

In [11]:
def llamada_API(url, coordenadas, query):
    parametros = {
    "client_id": tk_id,
    "client_secret": tk_secret,
    "v": "20180323",
    "ll": f"{coordenadas}",
    "query": f"{query}",
    "limit": 200  
}
    response = requests.get(url, params=parametros).json()["response"]["groups"][0]["items"]
    return response

In [12]:
def escoger_datos(response):
    name = []
    type_ = []
    lat = []
    lng = []
    cc = []
    adress = []
    for i in response:
        name.append(i["venue"]["name"])
    for i in response:
        type_.append(i["venue"]["categories"][0]["name"])
    for i in response:
        lng.append(i["venue"]["location"]["lng"])
    for i in response:
        lat.append(i["venue"]["location"]["lat"])
    for i in response:
        cc.append(i["venue"]["location"]["cc"])
    for i in response:
        adress.append(i["venue"]["location"]["formattedAddress"][0])      
    response2 = [name , type_, cc, adress,  lat, lng ]

    return response2

In [13]:
def crear_df(response2):
    df = pd.DataFrame(response2).T
    df1 = df.rename(columns = {0:"Name", 1:"Type", 2:"Country", 3:"Adress", 4:"Latitude", 5:"Longitude", })
    return df1

In [14]:
def get_geometry(df):
    df2 = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.Latitude, df.Longitude))
    return df2

In [15]:
def get_loc(df):
    df3 = df['geometry'].apply(lambda x:shapely.geometry.mapping(x))
    return pd.DataFrame(df3)

In [16]:
def df_final1(url, coordenadas, query):
    a  = llamada_API(url, coordenadas, query)
    b = escoger_datos(a)
    c = crear_df(b)
    d = get_geometry(c)
    d["location"] = get_loc(d)
    df_terminado = d[["Name","Type","Country", "Adress","Latitude","Longitude","location"]]
    return df_terminado

# FUNCTIONS TO GET CONDITIONS OF A LOCATION

# SCHOOLS

In [17]:
Schools = ["Nursery School","Elementary School","Middle School","High School"]
def get_schools(location, Schools):
    Nursery_sch = df_final1(url, location, Schools[0])
    Elementary_sch = df_final1(url, location, Schools[1])
    Middle_sch = df_final1(url, location, Schools[2])
    High_sch = df_final1(url, location, Schools[3])
    all_schools_bad_index = pd.concat([Nursery_sch, Elementary_sch, Middle_sch, High_sch])
    all_schools = all_schools_bad_index.reset_index(drop=True)
    all_schools.insert(1, "Classification", "School")
    return all_schools

# DESIGNING COMPANIES

In [18]:
def get_design_studio(location):
    DesignCo = df_final1(url, location, "Design Studio")
    DesignCo.insert(1, "Classification", "Designing Studios")
    return DesignCo


# TECH STARTUPS

In [19]:
def get_tech_startups(location):
    TechStaUp = df_final1(url, location, "Tech Startup")
    TechStaUp.insert(1, "Classification", "Technological Startups")
    return TechStaUp

    

# LEISURE

In [20]:
leisure = ["Nightclub", "Piano Bar", "Pub", "Irish Pub"]
def get_leisure(location, leisure):
    Disco = df_final1(url, location, leisure[0])
    Piano_bar = df_final1(url, location, leisure[1])
    Pub = df_final1(url, location, leisure[2])
    irish_pub = df_final1(url, location, leisure[3])
    all_leisure_bad_index = pd.concat([Disco, Piano_bar, Pub, irish_pub])
    leisure = all_leisure_bad_index.reset_index(drop=True)
    leisure.insert(1, "Classification", "Leisure Places")
    return leisure

# TRAVELLING STATIONS

In [21]:
stations = ["Metro Station", "Light Rail Station", "Bus Station"  ]
def get_closeby_stations(location, stations):
    Metro = df_final1(url, location, stations[0])
    Rail_Station = df_final1(url, location, stations[1])
    Bus_Station = df_final1(url, location, stations[1])
    all_stations_bad_index = pd.concat([Metro, Rail_Station, Bus_Station])
    stations = all_stations_bad_index.reset_index(drop=True)
    stations.insert(1, "Classification", "Travelling Stations")
    return stations

# PARKS

In [22]:
def get_parks(location):
    Parks = df_final1(url, london_loc, "Park")
    Parks.insert(1, "Classification", "Parks")
    return Parks

# DATAFRAME GLOBAL Y LIMPIO

In [23]:
def df_FINAL(location):
    a = get_schools(location, Schools)
    b = get_design_studio(location)
    c = get_tech_startups(location)
    d = get_leisure(location, leisure)
    e = get_closeby_stations(location,stations)
    f = get_parks(location)
    global_dataframe_bad_index = pd.concat([a,b,c,d,e,f])
    global_df_final = global_dataframe_bad_index.reset_index(drop=True)
    return global_df_final

In [24]:
a = df_FINAL(london_loc)

In [25]:
a.sample(10)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
114,FOODit - TechHub Old Street,Technological Startups,Tech Startup,GB,207 Old Street,51.525629,-0.088288,"{'type': 'Point', 'coordinates': (51.525628709..."
38,Notre Dame RC School,School,High School,GB,118 St. George's road,51.497096,-0.106522,"{'type': 'Point', 'coordinates': (51.497095663..."
256,The Windmill,Leisure Places,Pub,GB,27 Tabernacle St,51.523298,-0.086322,"{'type': 'Point', 'coordinates': (51.523298214..."
58,Chroma Studios,Designing Studios,Design Studio,GB,86-90 Paul Street,51.525719,-0.083741,"{'type': 'Point', 'coordinates': (51.525718702..."
127,OpenGamma HQ,Technological Startups,Tech Startup,GB,185 Park Street,51.513577,-0.080466,"{'type': 'Point', 'coordinates': (51.513576767..."
299,Bank London Underground and DLR Station,Travelling Stations,Light Rail Station,GB,Princes St (at Queen Victoria St),51.513376,-0.088931,"{'type': 'Point', 'coordinates': (51.513376461..."
267,The Gable,Leisure Places,Pub,GB,25 Moorgate,51.515804,-0.089487,"{'type': 'Point', 'coordinates': (51.515803817..."
279,Philomena's,Leisure Places,Irish Pub,GB,40 Great Queens St,51.514903,-0.12181,"{'type': 'Point', 'coordinates': (51.514903353..."
182,The Old Blue Last,Leisure Places,Pub,GB,39 Great Eastern St (at Curtain Rd),51.52437,-0.08027,"{'type': 'Point', 'coordinates': (51.524370284..."
199,The Reliance,Leisure Places,Pub,GB,336 Old St (at Charlotte Rd),51.526768,-0.081407,"{'type': 'Point', 'coordinates': (51.526767825..."


# LONDON

In [26]:
london_schools = get_schools(london_loc, Schools)
london_designing_companies = get_design_studio(london_loc)
london_tech_startups = get_tech_startups(london_loc)
london_leisure = get_leisure(london_loc, leisure)
london_travelling_stations = get_closeby_stations(london_loc, stations)
london_parks = get_parks(london_loc)
london_df_entero = df_FINAL(london_loc)

In [36]:
london_df_entero.sample(10)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
322,St Paul's Churchyard,Parks,Park,GB,St. Paul's Churchyard,51.51345,-0.097171,"{'type': 'Point', 'coordinates': (51.513449906..."
153,Cargo,Leisure Places,Nightclub,GB,83 Rivington St,51.526377,-0.07892,"{'type': 'Point', 'coordinates': (51.526376647..."
241,The Tokenhouse,Leisure Places,Pub,GB,4 Moorgate,51.515241,-0.089245,"{'type': 'Point', 'coordinates': (51.515240982..."
89,Agitprop,Designing Studios,Design Studio,GB,"19 Links Yard, (29a Spelman Street)",51.519353,-0.07005,"{'type': 'Point', 'coordinates': (51.519352818..."
279,Philomena's,Leisure Places,Irish Pub,GB,40 Great Queens St,51.514903,-0.12181,"{'type': 'Point', 'coordinates': (51.514903353..."
196,The Pride of Spitalfields,Leisure Places,Pub,GB,3 Heneage St (Brick Ln),51.51887,-0.071136,"{'type': 'Point', 'coordinates': (51.518869597..."
210,The Crosse Keys (Wetherspoon),Leisure Places,Pub,GB,9 Gracechurch St.,51.512685,-0.084751,"{'type': 'Point', 'coordinates': (51.512684501..."
342,Trinity Square Gardens,Parks,Garden,GB,Trinity Sq,51.509861,-0.077457,"{'type': 'Point', 'coordinates': (51.509860939..."
49,SCP,Designing Studios,Furniture / Home Store,GB,135-139 Curtain Rd.,51.526324,-0.080406,"{'type': 'Point', 'coordinates': (51.526324409..."
93,Lastminute print,Designing Studios,Design Studio,GB,29-31 Greatorex Street,51.518371,-0.067567,"{'type': 'Point', 'coordinates': (51.5183709, ..."


# SINGAPORE

In [28]:
singapore_schools = get_schools(singapore_loc, Schools)
singapore_designing_companies = get_design_studio(singapore_loc)
singapore_tech_startups = get_tech_startups(singapore_loc)
singapore_leisure = get_leisure(singapore_loc, leisure)
singapore_travelling_stations = get_closeby_stations(singapore_loc, stations)
singapore_parks = get_parks(singapore_loc)
singapore_df_entero = df_FINAL(singapore_loc)

In [35]:
singapore_df_entero.sample(10)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
176,QUEENS The Club,Leisure Places,Nightclub,SG,ESPLANADE,1.2896,103.856622,"{'type': 'Point', 'coordinates': (1.2896000532..."
361,Tower Hill,Parks,Neighborhood,GB,London,51.510053,-0.076536,"{'type': 'Point', 'coordinates': (51.510052558..."
250,O+ Victoria,Leisure Places,Pub,SG,32 Mosque St,1.283495,103.845285,"{'type': 'Point', 'coordinates': (1.2834948266..."
294,The Chamber,Leisure Places,Pub,SG,Robertson Walk,1.291756,103.841329,"{'type': 'Point', 'coordinates': (1.2917564105..."
93,River Valley Building,Technological Startups,Office,SG,179 River Valley Rd,1.292004,103.843694,"{'type': 'Point', 'coordinates': (1.2920037002..."
240,Aurora Boat Quay,Leisure Places,Pub,SG,Singapore,1.288638,103.849351,"{'type': 'Point', 'coordinates': (1.288638, 10..."
166,EZ50,Leisure Places,Nightclub,SG,1 Neil Rd #01-01,1.280281,103.844012,"{'type': 'Point', 'coordinates': (1.2802809408..."
232,Shin Bar,Leisure Places,Pub,SG,Boat Quay,1.287221,103.848126,"{'type': 'Point', 'coordinates': (1.287221, 10..."
155,Trace,Leisure Places,Nightclub,SG,5 Magazine Road #01-03,1.288744,103.843015,"{'type': 'Point', 'coordinates': (1.2887437803..."
175,Society Bar,Leisure Places,Nightclub,SG,"#01-11, One Fullerton (1 Fullerton Rd)",1.286561,103.854387,"{'type': 'Point', 'coordinates': (1.2865613739..."


# SAN FRANCISCO

In [29]:
sanfrancisco_schools = get_schools(san_francisco_loc, Schools)
sanfrancisco_designing_companies = get_design_studio(san_francisco_loc)
sanfrancisco_tech_startups = get_tech_startups(san_francisco_loc)
sanfrancisco_leisure = get_leisure(san_francisco_loc, leisure)
sanfrancisco_travelling_stations = get_closeby_stations(san_francisco_loc, stations)
sanfrancisco_parks = get_parks(san_francisco_loc)
sanfrancisco_df_entero = df_FINAL(san_francisco_loc)

In [34]:
sanfrancisco_df_entero.sample(10)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
4,Kids by the Bay - Civic Center,School,Preschool,US,"90 7th Street, Floor 1 Ste 2000 (Stevenson)",37.77943,-122.411819,"{'type': 'Point', 'coordinates': (37.779430009..."
96,Atelier Gabriella Daher,Designing Studios,Design Studio,US,Green St. (Polk St.),37.797764,-122.422242,"{'type': 'Point', 'coordinates': (37.797763552..."
294,Powell MUNI Metro Station,Travelling Stations,Metro Station,US,899 Market St (at Powell St),37.784414,-122.407949,"{'type': 'Point', 'coordinates': (37.784413539..."
7,Jean Parker Elementary,School,Elementary School,US,840 Broadway,37.797641,-122.411043,"{'type': 'Point', 'coordinates': (37.797641, -..."
47,Jewish Community High School of the Bay,School,High School,US,1835 Ellis St (Pierce),37.781714,-122.436442,"{'type': 'Point', 'coordinates': (37.781714330..."
79,Beast,Designing Studios,Design Studio,US,500 Sansome St,37.795513,-122.401277,"{'type': 'Point', 'coordinates': (37.795512549..."
0,Kids by the Bay - Financial District,School,Daycare,US,75 Hawthorne St,37.785471,-122.398037,"{'type': 'Point', 'coordinates': (37.785471, -..."
267,The Irish Bank,Leisure Places,Irish Pub,US,10 Mark Ln (at Harlan Pl.),37.790495,-122.404759,"{'type': 'Point', 'coordinates': (37.790495126..."
251,Cresta's Twenty Two Eleven Club,Leisure Places,Pub,US,2211 Polk St (btw Bonita St & Vallejo St),37.7971,-122.422061,"{'type': 'Point', 'coordinates': (37.797100262..."
37,Galileo Academy of Science & Technology,School,High School,US,1150 Francisco St (Van Ness Ave.),37.804105,-122.424195,"{'type': 'Point', 'coordinates': (37.804104599..."


In [42]:
sanfrancisco_df_entero.to_csv("Data/SanFrancisco.csv")

In [43]:
london_df_entero.to_csv("Data/London.csv")

In [44]:
singapore_df_entero.to_csv("Data/Singapore.csv")